# Aplicação prática das Transformers
## Tradução automática de texto de Inglês para Português

In [1]:
file_path="ParaCrawl99k/ParaCrawl99K_EnPt_PCrawlGoogleT.pkl"

In [2]:
import pickle
import random

with open(file_path, "rb") as f:
        dataset = pickle.load(f)

random.shuffle(dataset)

In [3]:
import pickle
import random

with open(file_path, "rb") as f:
        dataset = pickle.load(f)

random.shuffle(dataset)

In [ ]:
!pip install Transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 112.0 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.1 MB/s eta 0:00:00


In [ ]:
import transformers 

## ATIVIDADE 1

Separe o conjunto de dados ParaCrawl99k em 80% para treino, 10% para validação e 10% para teste.

In [5]:
# Embaralha os dados e divide em 80% para treino, 10% para validação e 10% para teste
nData = len(dataset)

nTrain = int(nData * 0.8)  
nVal = int(nData * 0.1)  
train = dataset[:nTrain]
val = dataset[(nTrain + 1):(nTrain + nVal)]
test = dataset[(nTrain + nVal + 1): (nTrain + 2 * nVal)]

In [7]:
!pip install -q transformers datasets


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: C:\Users\Ricardo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
from datasets import Dataset

C:\Users\Ricardo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
def create_dataset(data):
  for index, element in enumerate(data):
    yield {'id': index, 'translation': {'en': element[0], 'pt': element[1]}}

train_dataset = Dataset.from_generator(create_dataset, gen_kwargs={"data": train})
val_dataset = Dataset.from_generator(create_dataset, gen_kwargs={"data": val})
test_dataset = Dataset.from_generator(create_dataset, gen_kwargs={"data": test})

Dataset generator downloaded and prepared to C:/Users/Ricardo/.cache/huggingface/datasets/generator/default-7b69baed7eab952d/0.0.0. Subsequent calls will reuse this data.


Dataset generator downloaded and prepared to C:/Users/Ricardo/.cache/huggingface/datasets/generator/default-621bbcb40bf79e60/0.0.0. Subsequent calls will reuse this data.


Dataset generator downloaded and prepared to C:/Users/Ricardo/.cache/huggingface/datasets/generator/default-ebc9b1b71ff4f667/0.0.0. Subsequent calls will reuse this data.


In [10]:
# Visualizando a estrutura do conjunto de treino
train_dataset

Dataset({
    features: ['id', 'translation'],
    num_rows: 79269
})

In [11]:
# Visualizando a estrutura do conjunto de validação
val_dataset

Dataset({
    features: ['id', 'translation'],
    num_rows: 9907
})

In [12]:
# Visualizando a estrutura do conjunto de teste
test_dataset

Dataset({
    features: ['id', 'translation'],
    num_rows: 9907
})

In [13]:
#visualizando um elemento do conjunto de treino
train_dataset[20]

{'id': 20,
 'translation': {'en': 'But we do know certain things about Judas as fact: that Jesus washed his feet, let him dip his bread in His own cup, and made him His treasurer.',
  'pt': 'Mas sabemos certas coisas sobre Judas como fatos: que Jesus lavou os pés, mergulhou o pão em sua própria taça e fez dele seu tesoureiro.'}}

## ATIVIDADE 2 

Utilizando a biblioteca Huggingface, carregue o tokenizador referente ao modelo T5 e quebre os textos em Inglês e em Português em tokens. Note que como estamos utilizando um modelo T5, ele espera um prefixo que indique a tarefa a ser realizada. Então, adicione ao início dos textos em Inglês o prefixo “translate English to Portuguese”.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [ ]:
tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
       print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
source_lang = "en"
target_lang = "pt"
prefix = "translate English to Portuguese: " 

def preprocess_function(examples):
  inputs = [prefix + example[source_lang] for example in examples["translation"]]
  targets = [example[target_lang] for example in examples["translation"]]
  model_inputs = tokenizer(inputs, max_length=128, truncation=True)

  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=128, truncation=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
train_tokenized_sentences = train_dataset.map(preprocess_function, batched=True)
val_tokenized_sentences = val_dataset.map(preprocess_function, batched=True)
test_tokenized_sentences = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/79269 [00:00<?, ? examples/s]

Map:   0%|          | 0/9907 [00:00<?, ? examples/s]

## ATIVIDADE 3

Utilizando o modelo pré-treinado, sem fine tuning, calcule a métrica BLEU para o conjunto de validação. Este será seu baseline.

In [ ]:
from transformers import pipeline

en_pt_translator = pipeline(f"translation_{source_lang}_to_{target_lang}", model="t5-small", tokenizer=tokenizer,max_length=400)

In [ ]:
# Sorteando uma amostra de 5 sentenças para testando as predições no conjunto de validação
import numpy as np

for val_sample in np.random.choice(val_dataset, size=5):
  original = val_sample['translation'][source_lang]
  target_text = val_sample['translation'][target_lang]
  translation = en_pt_translator(original)
  print(f"Original Text: {original} \n Prediction: {translation[0]['translation_text']} \n Answer: {target_text}")

Original Text: Provence - a style, combining elegance, comfort and simplicity. 
 Prediction: Provence - style, combining elegance, comfort and simplicity. 
 Answer: Provence - um estilo, combinando elegância, conforto e simplicidade.
Original Text: the parallelepided tool with possibility of simple editing — for creation of parallelepipeds in the long term; 
 Prediction: with the possibility of simple editing — for the creation of parallelepipeds in long term; 
 Answer: a ferramenta paralelepípedo com possibilidade de edição simples - para criação de paralelepípedos a longo prazo;
Original Text: We liked the pasta risotto, that although he was at the point, We have eaten better. So, ask that the next day another different pasta, since there was a variety (noodles with black, ravioli, taglioni, Sorrento, etc.) and I tell you. 
 Prediction: ,,, and ask that the next day another different pasta, since there was a variety (noodles with black, ravioli, taglioni, Sorrento, etc.) and I tell y

In [ ]:
import numpy as np

val_dataset_sample = np.random.choice(val_dataset["translation"], size=len(val_dataset))
references = [sample[target_lang] for sample in val_dataset_sample]
translations = [en_pt_translator(sample[source_lang])[0]['translation_text'] for sample in val_dataset_sample]

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.2 MB/s eta 0:00:00


In [ ]:
import evaluate

In [ ]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=translations, references=references)
print(results)

{'bleu': 0.04528627105227495, 'precisions': [0.16698367636621717, 0.051263858093126384, 0.028846873747583247, 0.017032648592955116], 'brevity_penalty': 1.0, 'length_ratio': 1.0730636331195111, 'translation_length': 246575, 'reference_length': 229786}


## ATIVIDADE 4 
Faça o ajuste fino (fine tuning) do modelo utilizando o conjunto de treino definido anterior- mente. Para este experimento, realize o treinamento utilizando dois valores distintos para a taxa de aprendizado (learning rate): 0.01 e 0.00001. Quais foram o valores da métrica BLEU obtidos no conjunto de validação em cada um dos casos? Discuta os resultados elaborando hipóteses para a diferença em termos de performance.

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-small")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

tf_train_set = train_tokenized_sentences.to_tf_dataset(columns=["attention_mask", "input_ids", "labels"],shuffle=True, batch_size=64,
        collate_fn=data_collator)

tf_val_set = val_tokenized_sentences.to_tf_dataset(columns=["attention_mask", "input_ids", "labels"],shuffle=False,
            batch_size=64,
            collate_fn=data_collator)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Para taxa de aprendizado (learning rate) de 0.01

In [ ]:
from transformers import create_optimizer, AdamWeightDecay
optimizer = AdamWeightDecay(learning_rate=0.01, weight_decay_rate=0.01)

In [ ]:
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
model.fit(x=tf_train_set, validation_data=tf_val_set, epochs=3)

Epoch 1/3
1239/1239 [==============================] - 483s 372ms/step - loss: 1.3234 - val_loss: 0.7287
Epoch 2/3
1239/1239 [==============================] - 457s 369ms/step - loss: 0.8227 - val_loss: 0.6153
Epoch 3/3
1239/1239 [==============================] - 457s 369ms/step - loss: 0.7205 - val_loss: 0.5736


In [ ]:
from transformers import pipeline

en_pt_translator = pipeline(f"translation_{source_lang}_to_{target_lang}", model=model, tokenizer=tokenizer,max_length=400)

In [ ]:
#Testando as predições
# Sorteando uma amostra de 5 sentenças no conjunto de validação
import numpy as np

for val_sample in np.random.choice(val_dataset, size=5):
  original = val_sample['translation'][source_lang]
  target_text = val_sample['translation'][target_lang]
  translation = en_pt_translator(original)
  print(f"Original Text: {original} \n Prediction: {translation[0]['translation_text']} \n Answer: {target_text}")

Original Text: It features a spacious outdoor terrace and a renowned restaurant. 
 Prediction: Apresenta um terraço ao ar espaçoso terraço ao ar livre e um restaurante renomado. 
 Answer: Possui um espaçoso terraço ao ar livre e um restaurante de renome.
Original Text: Dr. ███: All right, I believe we are done here. Thank you, Captain. 
 Prediction: Todos certos, acredito que estamos feitos aqui. Obrigado, Captain. 
 Answer: Dr. ███: Tudo bem, acredito que terminamos aqui. Obrigado, capitão.
Original Text: Such is the measure of his understanding. 
 Prediction: Essa é a medida de seu entendimento. 
 Answer: Essa é a medida do seu entendimento.
Original Text: This pathology occurs with the growth of the anterior wall of the lower costal cartilage. 
 Prediction: Esta patologia ocorre com o crescimento da parede anterior da cartilagem inferior da cartilagem de custo inferior inferior. 
 Answer: Essa patologia ocorre com o crescimento da parede anterior da cartilagem costal inferior.
Origi

In [ ]:
import numpy as np

val_dataset_sample = np.random.choice(val_dataset["translation"], size=len(val_dataset))
references = [sample[target_lang] for sample in val_dataset_sample]
translations = [en_pt_translator(sample[source_lang])[0]['translation_text'] for sample in val_dataset_sample]

In [ ]:
import evaluate

In [ ]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=translations, references=references)
print(results)

{'bleu': 0.5598847253740015, 'precisions': [0.8426966292134831, 0.6785714285714286, 0.5443037974683544, 0.43243243243243246], 'brevity_penalty': 0.9243618371081701, 'length_ratio': 0.9270833333333334, 'translation_length': 89, 'reference_length': 96}


Para taxa de aprendizado (learning rate) de 0.00001

In [ ]:
from transformers import create_optimizer, AdamWeightDecay
optimizer = AdamWeightDecay(learning_rate=0.00001, weight_decay_rate=0.00001)

In [ ]:
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
model.fit(x=tf_train_set, validation_data=tf_val_set, epochs=3)

Epoch 1/3
1239/1239 [==============================] - 1401s 1s/step - loss: 0.6123 - val_loss: 0.5439
Epoch 2/3
1239/1239 [==============================] - 1382s 1s/step - loss: 0.5854 - val_loss: 0.5334
Epoch 3/3
1239/1239 [==============================] - 1381s 1s/step - loss: 0.5688 - val_loss: 0.5254


In [ ]:
en_pt_translator = pipeline(f"translation_{source_lang}_to_{target_lang}", model=model, tokenizer=tokenizer)

In [ ]:
# Testando as predições do modelo com learning rate de 0.00001

In [ ]:
# Sorteando uma amostra de 5 sentenças para testando as predições no conjunto de validação
import numpy as np

for val_sample in np.random.choice(val_dataset, size=5):
  original = val_sample['translation'][source_lang]
  target_text = val_sample['translation'][target_lang]
  translation = en_pt_translator(original)
  print(f"Original Text: {original} \n Prediction: {translation[0]['translation_text']} \n Answer: {target_text}")

Your input_length: 20 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 36 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


Original Text: Lestes viridis (Vander Linden, 1825) - No validated 
 Prediction: Viridis (Vander Linden, 1825) - No valida 
 Answer: Lestes viridis (Vander Linden, 1825) - Não validado


Your input_length: 34 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


Original Text: Being subjected to mechanical forces, successful healing outcomes are dependent on the structural integrity of L-PRF and hence elucidating their physical properties is important. 
 Prediction: Os resultados de cura so dependentes da integridade 
 Answer: Sendo submetido a forças mecânicas, os resultados de cura bem-sucedidos dependem da integridade estrutural do L-PRF e, portanto, é importante elucidar suas propriedades físicas.


Your input_length: 20 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


Original Text: Sant’Agnello is a small town located in the Peninsula Sorrentina, Campania, between Meta and Sorrento. 
 Prediction: A Agnello é uma pequena cidade localiza 
 Answer: Sant&#39;Agnello é uma pequena cidade localizada na Península Sorrentina, Campania, entre Meta e Sorrento.
Original Text: By 2003 the company was fully acquired and is now called Laerdal Sophus. 
 Prediction: Agora a empresa foi totalmente aquista 
 Answer: Em 2003, a empresa foi totalmente adquirida e agora se chama Laerdal Sophus.
Original Text: Wi-Fi access is available in this holiday park. 
 Prediction: O acesso Wi-Fi está disponvel neste parque 
 Answer: O acesso Wi-Fi está disponível neste parque de férias.


In [ ]:
import numpy as np

val_dataset_sample = np.random.choice(val_dataset["translation"], size=len(val_dataset))
references = [sample[target_lang] for sample in val_dataset_sample]
translations = [en_pt_translator(sample[source_lang])[0]['translation_text'] for sample in val_dataset_sample]

In [ ]:
import evaluate

In [ ]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=translations, references=references)
print(results)